In [27]:
# yyyy-mm-dd
def loaddata(from_date, to_date):
    from datetime import datetime
    # Import necessary modules
    from ssi_fc_data import fc_md_client, model
    import pandas as pd  # Import Pandas for DataFrame handling
    import json
    import configDataSSL

    # Create a Market Data Client
    # from_date = "01/11/2023"
    # to_date = "17/11/2023"

    # Chuyen tu yyyy-mm-dd sang dd/mm/yyyy cho phu hop SSI
    # Chuyển đổi chuỗi thành đối tượng datetime
    from_date = datetime.strptime(from_date, '%Y-%m-%d')
    to_date = datetime.strptime(to_date, '%Y-%m-%d')

    # Chuyển đổi đối tượng datetime sang định dạng 'dd/mm/yyyy'
    from_date = from_date.strftime('%d/%m/%Y')
    to_date = to_date.strftime('%d/%m/%Y')

    client = fc_md_client.MarketDataClient(configDataSSL)

    req = model.daily_ohlc('VCB', from_date, to_date) # Viet lai lay tu yfinance

    data_dict = client.daily_ohlc(configDataSSL, req)
  
    # Access the list of dictionaries in the "data" field
    data_list = data_dict['data']

    # Convert the list of dictionaries into a DataFrame
    data = pd.DataFrame(data_list)

    data = data.rename(columns={'TradingDate': 'Datetime'})
    # Chuyển đổi cột 'Datetime' từ 'dd/mm/yyyy' sang đối tượng datetime
    data['Datetime'] = pd.to_datetime(data['Datetime'], format='%d/%m/%Y')

    # Định dạng lại cột 'Datetime' sang 'yyyy-mm-dd'
    data['Datetime'] = data['Datetime'].dt.strftime('%Y-%m-%d')

    # Print or work with the DataFrame

    return data

In [28]:
import yfinance as yf

def loaddataYFinance(from_date, to_date):
    data = yf.download('VCB.VN', start=from_date, end=to_date)
    data.reset_index(inplace=True)
    data = data.rename(columns={'Date': 'Datetime'})
    
    return data

In [29]:
import pandas as pd

Entry = loaddataYFinance('2023-11-10', '2023-11-20')
ohlc_df = loaddataYFinance('2023-11-15', '2023-11-20')

print(Entry)
# Ta co dataframe Entry
Entry['Datetime'] = pd.to_datetime(Entry['Datetime'])
# Ta co dataframe ohlc_df

def find_pos_ohl(time, ohlc_df):
    # Ensure that 'time' is a pandas Timestamp
    time = pd.to_datetime(time)

    # Make sure the DataFrame's index is a DatetimeIndex
    if not isinstance(ohlc_df.index, pd.DatetimeIndex):
        ohlc_df = ohlc_df.set_index(pd.to_datetime(ohlc_df.index))

    # Use searchsorted to find the index. This will not raise a KeyError
    # if the exact match is not found. Instead, it returns an index where
    # the time would be inserted to maintain order.
    pos = ohlc_df.index.searchsorted(time, side='left')

    # Adjust position if it's out of bounds
    if pos >= len(ohlc_df.index):
        pos = len(ohlc_df.index) - 1
    elif pos > 0 and (time - ohlc_df.index[pos-1]) < (ohlc_df.index[pos] - time):
        pos -= 1  # Use the closer time

    return pos

    pos = ohlc_df.index.get_loc(time)
    return pos

Entry['Pos_ohl'] = Entry['Datetime'].apply(lambda x: find_pos_ohl(x, ohlc_df))

print(Entry)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
    Datetime     Open     High      Low    Close  Adj Close   Volume
0 2023-11-10  87100.0  87500.0  86000.0  86000.0    86000.0  1411000
1 2023-11-13  86000.0  86100.0  85400.0  85500.0    85500.0  3242786
2 2023-11-14  86000.0  86500.0  85400.0  86000.0    86000.0  1086500
3 2023-11-15  86500.0  87700.0  86500.0  87700.0    87700.0  4172800
4 2023-11-16  87000.0  88000.0  86700.0  88000.0    88000.0  1343800
5 2023-11-17  87800.0  87900.0  85500.0  85600.0    85600.0  1693000
    Datetime     Open     High      Low    Close  Adj Close   Volume  Pos_ohl
0 2023-11-10  87100.0  87500.0  86000.0  86000.0    86000.0  1411000        2
1 2023-11-13  86000.0  86100.0  85400.0  85500.0    85500.0  3242786        2
2 2023-11-14  86000.0  86500.0  85400.0  86000.0    86000.0  1086500        2
3 2023-11-15  86500.0  87700.0  86500.0  87700.0    87700.0  417280